In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import numpy as np
from sklearn.impute import SimpleImputer
import mlflow
import mlflow.sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

In [2]:
from azure.storage.filedatalake import DataLakeServiceClient
import os
from dotenv import load_dotenv


# Load environment variables from .env file
load_dotenv('notebooks/corrected/.env')

# Replace with your details
storage_account_name = "mldebugdevadls"
storage_account_key = os.getenv('AZURE_STORAGE_KEY')

# Connect to ADLS
service_client = DataLakeServiceClient(
    account_url=f"https://{storage_account_name}.dfs.core.windows.net",
    credential=storage_account_key,
    api_version="2023-11-03"  # Use the correct supported API version
)

# List Containers
containers = service_client.list_file_systems()
for container in containers:
    print(container.name)


data


In [3]:
import os
from azure.storage.blob import BlobServiceClient
import pandas as pd
import io

def read_csv_from_blob(storage_account_name, container_name, file_name, storage_account_key=None):
    """
    Read a CSV file from Azure Blob Storage using Python and return a Pandas DataFrame.

    :param storage_account_name: Azure storage account name.
    :param container_name: Blob container name.
    :param file_name: Name of the file in the container.
    :param storage_account_key: Storage account access key.
    :return: Pandas DataFrame.
    """
    
    if not storage_account_key:
        # Try to get the key from environment variables if not provided
        storage_account_key = os.environ.get('AZURE_STORAGE_KEY')
        
    if not storage_account_key:
        raise ValueError("Storage account key must be provided either as a parameter or as an environment variable 'AZURE_STORAGE_KEY'")
    
    try:
        # Create a connection string
        connection_string = f"DefaultEndpointsProtocol=https;AccountName={storage_account_name};AccountKey={storage_account_key};EndpointSuffix=core.windows.net"
        
        # Create the BlobServiceClient
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        
        # Get the container client
        container_client = blob_service_client.get_container_client(container_name)
        
        # Get the blob client
        blob_client = container_client.get_blob_client(file_name)
        
        # Download the blob content
        download_stream = blob_client.download_blob()
        
        # Convert the content to a DataFrame
        content = download_stream.readall()
        df = pd.read_csv(io.BytesIO(content))
        
        return df
    
    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None

In [4]:
training_data = read_csv_from_blob(storage_account_name=storage_account_name,
                                      container_name="data", 
                                      file_name="PreProcessingfinaldata.csv",
                                      storage_account_key=storage_account_key)
training_data.head()

,Unnamed: 0,week,sales_amount,base_price,final_price,promotion_type,facebook_spend,google ads_spend,influencer marketing_spend,instagram_spend,ooh_spend,print_spend,radio_spend,tv_spend,youtube_spend,facebook_ctr,google ads_ctr,influencer marketing_ctr,instagram_ctr,youtube_ctr
0,0,2023-01-01,13516527.77,101.830513,94.148539,Percentage Discount,1152.82,810.68,1122.12,707.48,11230.47,6214.43,6723.33,11311.42,703.13,3.047174,4.333516,2.471559,2.008197,2.116972
1,1,2023-01-08,95081753.02,101.830513,94.148539,Percentage Discount,7472.53,6973.85,7179.02,6834.33,11380.75,11069.60,9505.50,14004.01,6562.00,2.636847,2.732868,2.930657,3.354279,3.570124
2,2,2023-01-15,94804406.04,101.830513,94.148539,Percentage Discount,7204.29,7383.50,7185.08,6963.40,10270.30,8861.17,7836.04,14442.24,7318.19,2.679349,3.136116,2.879586,2.938546,3.776793
3,3,2023-01-22,94833974.28,101.830513,94.148539,Percentage Discount,7726.84,6522.20,7710.31,7479.41,8335.56,11601.91,6663.51,11917.89,7745.75,3.015955,3.836348,2.983655,2.858832,2.823088
4,4,2023-01-29,94806994.45,101.830513,94.148539,Percentage Discount,6987.22,6969.68,7094.25,7294.12,9575.21,7488.18,12158.00,6753.84,7044.42,2.744554,3.511152,2.338256,2.403631,3.019390


## Final Preparing data for training

In [5]:
# Select features for scaling and encoding
numerical_features = ['sales_amount', 'base_price', 'final_price',
                      'facebook_spend', 'google ads_spend', 'influencer marketing_spend',
                      'instagram_spend', 'ooh_spend', 'print_spend', 'radio_spend',
                      'tv_spend', 'youtube_spend', 'facebook_ctr', 'google ads_ctr',
                      'influencer marketing_ctr', 'instagram_ctr', 'youtube_ctr']

categorical_features = ['promotion_type']

In [6]:
# Step 1: Scale numerical features
scaler = StandardScaler()
scaled_numerical_data = scaler.fit_transform(training_data[numerical_features])

# Convert scaled numerical data to a DataFrame
scaled_numerical_df = pd.DataFrame(scaled_numerical_data, columns=numerical_features)

# Step 2: Encode categorical features
encoder = OneHotEncoder(sparse_output=False)
encoded_categorical_data = encoder.fit_transform(training_data[categorical_features])

# Get column names for encoded categorical features
encoded_categorical_columns = encoder.get_feature_names_out(categorical_features)

# Convert encoded categorical data to a DataFrame
encoded_categorical_df = pd.DataFrame(encoded_categorical_data, columns=encoded_categorical_columns)

# Step 3: Combine scaled numerical data and encoded categorical data
processed_df = pd.concat([scaled_numerical_df, encoded_categorical_df], axis=1)

# Display the processed DataFrame
processed_df.head()

,sales_amount,base_price,final_price,facebook_spend,google ads_spend,influencer marketing_spend,instagram_spend,ooh_spend,print_spend,radio_spend,tv_spend,youtube_spend,facebook_ctr,google ads_ctr,influencer marketing_ctr,instagram_ctr,youtube_ctr,promotion_type_Buy One Get One Free,promotion_type_Percentage Discount
0,-7.208240,-1.888843,-1.079989,-6.138594,-6.036483,-6.258867,-6.414760,0.452694,-1.329475,-0.967204,0.269455,-6.263100,0.408029,2.567653,-1.354243,-2.366897,-2.237695,0.0,1.0
1,0.156877,-1.888843,-1.079989,0.671809,0.104967,0.342237,-0.124517,0.511463,0.407651,0.102746,1.192669,-0.319184,-0.548539,-0.479803,-0.315938,0.701320,1.459348,0.0,1.0
2,0.131833,-1.888843,-1.079989,0.382741,0.513173,0.348841,0.007995,0.077207,-0.382501,-0.539284,1.342926,0.447983,-0.449456,0.287937,-0.431442,-0.246289,1.985146,0.0,1.0
3,0.134503,-1.888843,-1.079989,0.945865,-0.345091,0.921263,0.537766,-0.679400,0.598106,-0.990209,0.477397,0.881749,0.335249,1.621101,-0.196076,-0.427986,-0.441227,0.0,1.0
4,0.132067,-1.888843,-1.079989,0.148817,0.100812,0.249850,0.347535,-0.194618,-0.873741,1.122828,-1.293211,0.170239,-0.297450,1.001963,-1.655725,-1.465558,0.058197,0.0,1.0


In [7]:
# Check if there are any zero or negative values in the numerical columns
print((processed_df[numerical_features] <= 0).sum())

sales_amount                   1
base_price                    27
final_price                   27
facebook_spend                24
google ads_spend              19
influencer marketing_spend    22
instagram_spend               19
ooh_spend                     28
print_spend                   28
radio_spend                   31
tv_spend                      24
youtube_spend                 24
facebook_ctr                  24
google ads_ctr                27
influencer marketing_ctr      31
instagram_ctr                 29
youtube_ctr                   27
dtype: int64


In [8]:
# Replace zero or negative values with a small positive value (e.g., 1e-6)
processed_df[numerical_features] = processed_df[numerical_features].applymap(lambda x: max(x, 1e-6))

# Now apply log1p to handle the log transformation safely
X_log = np.log1p(processed_df[numerical_features])


C:\Users\Omkar\AppData\Local\Temp\ipykernel_32872\1579213879.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  processed_df[numerical_features] = processed_df[numerical_features].applymap(lambda x: max(x, 1e-6))


In [9]:
processed_df

,sales_amount,base_price,final_price,facebook_spend,google ads_spend,influencer marketing_spend,instagram_spend,ooh_spend,print_spend,radio_spend,tv_spend,youtube_spend,facebook_ctr,google ads_ctr,influencer marketing_ctr,instagram_ctr,youtube_ctr,promotion_type_Buy One Get One Free,promotion_type_Percentage Discount
0,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.452694,0.000001,0.000001,0.269455,0.000001,0.408029,2.567653,0.000001,0.000001,0.000001,0.0,1.0
1,0.156877,0.000001,0.000001,0.671809,0.104967,0.342237,0.000001,0.511463,0.407651,0.102746,1.192669,0.000001,0.000001,0.000001,0.000001,0.701320,1.459348,0.0,1.0
2,0.131833,0.000001,0.000001,0.382741,0.513173,0.348841,0.007995,0.077207,0.000001,0.000001,1.342926,0.447983,0.000001,0.287937,0.000001,0.000001,1.985146,0.0,1.0
3,0.134503,0.000001,0.000001,0.945865,0.000001,0.921263,0.537766,0.000001,0.598106,0.000001,0.477397,0.881749,0.335249,1.621101,0.000001,0.000001,0.000001,0.0,1.0
4,0.132067,0.000001,0.000001,0.148817,0.100812,0.249850,0.347535,0.000001,0.000001,1.122828,0.000001,0.170239,0.000001,1.001963,0.000001,0.000001,0.058197,0.0,1.0
5,0.130609,0.330143,0.302810,0.119731,0.000001,0.000001,0.000001,0.853710,0.000001,0.000001,0.000001,0.000001,0.317811,0.813612,0.000001,0.000001,0.067670,0.0,1.0
6,0.179697,1.217737,0.855929,0.179648,0.000001,0.340624,0.352258,0.000001,0.000001,0.000001,0.425990,0.517001,0.443638,0.294262,0.388764,0.000001,0.000001,0.0,1.0
7,0.107579,1.217737,0.855929,1.070914,0.313539,0.020535,0.000001,0.000001,0.000001,1.652694,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.207362,0.0,1.0
8,0.138756,1.217737,0.855929,0.000001,1.143773,0.493802,0.536883,0.000001,0.764055,0.000001,0.860584,0.210130,0.541265,0.000001,0.233650,0.000001,0.000001,0.0,1.0
9,0.115822,0.865677,0.000001,0.713104,1.014600,0.000001,0.435849,0.237089,0.879707,1.833905,0.562141,0.000001,0.000001,0.000001,0.802325,0.000001,0.000001,0.0,1.0


## Linear Regression

In [12]:
# from azureml.core import Workspace

# ws = Workspace.from_config(path='config.json')

In [13]:
# # mlflow.set_tracking_uri("http://localhost:5000")
# from azureml.core import Workspace

# ws = Workspace.from_config(path='config.json')
# mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())


In [ ]:
! az login

^C


In [24]:
import os
import pandas as pd
import statsmodels.api as sm
import mlflow
import mlflow.statsmodels
from sklearn.metrics import mean_absolute_error
from mlflow.models.signature import infer_signature

mlflow.set_experiment("optimizing-ad-spend-experiment")

target_column = 'sales_amount'
X = processed_df.drop(columns=[target_column])
y = processed_df[target_column]

with mlflow.start_run(run_name="Linear_Regression_StatsModels"):
    X_const = sm.add_constant(X)
    model = sm.OLS(y, X_const).fit()
    y_pred = model.predict(X_const)
    r2 = model.rsquared
    mae = mean_absolute_error(y, y_pred)
    mlflow.log_metrics({
        "r_squared": r2,
        "mae": mae
    })
    mlflow.log_param("model_type", "statsmodels.OLS")
    mlflow.log_param("include_intercept", True)
    mlflow.log_param("n_features", X.shape[1])
    coef_df = pd.DataFrame({
        "feature": X_const.columns,
        "coefficient": model.params.values,
        "p_value": model.pvalues.values
    }).sort_values(by="coefficient", ascending=False)
    for _, row in coef_df.iterrows():
        feat = row["feature"]
        mlflow.log_metric(f"coef__{feat}", row["coefficient"])
        mlflow.log_metric(f"pval__{feat}", row["p_value"])
    os.makedirs("models", exist_ok=True)
    coef_csv = "models/coefficients_and_pvalues.csv"
    coef_df.to_csv(coef_csv, index=False)
    mlflow.log_artifact(coef_csv, artifact_path="coefficients")
    signature = infer_signature(X_const, y_pred)
    input_example = X_const.head(3)
    mlflow.statsmodels.log_model(
        model,
        artifact_path="model",
        registered_model_name="ad_spend_ols_model",  
        signature=signature,
        input_example=input_example
    )
    run_id = mlflow.active_run().info.run_id
    result = mlflow.register_model(f"runs:/{run_id}/model", "Linear Regression")
    print(f"Model registered as version {result.version}.") 
    print(" Run logged with R², MAE, coefficients + p-values, signature, and model artifact.")


2025/05/26 17:18:06 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Omkar\AppData\Local\Temp\tmpxu0val7d\model, flavor: statsmodels). Fall back to return ['statsmodels==0.14.4']. Set logging level to DEBUG to see the full traceback. 
2025/05/26 17:18:06 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Registered model 'ad_spend_ols_model' already exists. Creating a new version of this model...
Created version '4' of model 'ad_spend_ols_model'.


Model registered as version 2.
✅ Run logged with R², MAE, coefficients + p-values, signature, and model artifact.


Registered model 'Linear Regression' already exists. Creating a new version of this model...
Created version '2' of model 'Linear Regression'.


## log-log regression training

In [25]:
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error
import mlflow
import mlflow.sklearn

mlflow.set_experiment("optimizing-ad-spend-experiment")

target_column = 'sales_amount'
X = processed_df.drop(columns=[target_column])
y = processed_df[target_column]

X_log = np.log1p(X)
y_log = np.log1p(y)

with mlflow.start_run(run_name="Log-Log Linear Regression"):
    model = LinearRegression()
    model.fit(X_log, y_log)

    y_pred_log = model.predict(X_log)
    r2 = r2_score(y_log, y_pred_log)
    mae_log = mean_absolute_error(y_log, y_pred_log)

    mlflow.log_metrics({
        "r_squared": r2,
        "mae_log": mae_log
    })
    mlflow.log_params({
        "model_type": "LinearRegression",
        "transformation": "log1p",
        "n_features": X.shape[1]
    })

    coef_df = (
        pd.DataFrame({
            "feature": X_log.columns,
            "coefficient": model.coef_
        })
        .sort_values(by="coefficient", ascending=False)
    )
    for _, row in coef_df.iterrows():
        mlflow.log_metric(f"coef__{row['feature']}", row["coefficient"])

    os.makedirs("models", exist_ok=True)
    coef_path = "models/coefficients.csv"
    coef_df.to_csv(coef_path, index=False)
    mlflow.log_artifact(coef_path, artifact_path="coefficients")

    mlflow.sklearn.log_model(
        model,
        artifact_path="model",
        registered_model_name="LogLog_Linear_v1"
    )

    run_id = mlflow.active_run().info.run_id
    mv = mlflow.register_model(f"runs:/{run_id}/model", "LogLog_Linear")
    mlflow.set_tag("model_version", mv.version)
    mlflow.log_param("model_version", mv.version)


2025/05/26 17:19:31 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Omkar\AppData\Local\Temp\tmpv5lr7bd_\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.6.1', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 
2025/05/26 17:19:31 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/05/26 17:19:39 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Omkar\AppData\Local\Temp\tmp0o6dntd5\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.6.1', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 
2025/05/26 17:19:39 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without

## Lasso Regression

In [26]:
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score, mean_absolute_error
import mlflow
import mlflow.sklearn

mlflow.set_experiment("optimizing-ad-spend-experiment")

# assuming X_log and y_log are already defined as in your previous code
with mlflow.start_run(run_name="Lasso_Regression_Log-Log"):
    alpha = 0.1
    lasso_model = Lasso(alpha=alpha)
    lasso_model.fit(X_log, y_log)

    y_pred = lasso_model.predict(X_log)
    r2 = r2_score(y_log, y_pred)
    mae = mean_absolute_error(y_log, y_pred)
    mlflow.log_metrics({"r_squared": r2, "mae_log": mae})

    mlflow.log_params({
        "model_type": "Lasso",
        "alpha": alpha,
        "transformation": "log1p",
        "n_features": X_log.shape[1]
    })

    coef_df = (
        pd.DataFrame({
            "feature": X_log.columns,
            "coefficient": lasso_model.coef_
        })
        .sort_values(by="coefficient", ascending=False)
    )
    for _, row in coef_df.iterrows():
        mlflow.log_metric(f"coef__{row['feature']}", row["coefficient"])

    os.makedirs("models", exist_ok=True)
    coef_path = "models/lasso_coefficients.csv"
    coef_df.to_csv(coef_path, index=False)
    mlflow.log_artifact(coef_path, artifact_path="coefficients")

    mlflow.sklearn.log_model(
        lasso_model,
        artifact_path="model",
        registered_model_name="Lasso_LogLog_Model"
    )

    run_id = mlflow.active_run().info.run_id
    mv = mlflow.register_model(f"runs:/{run_id}/model", "Lasso_LogLog_Model")
    mlflow.set_tag("model_version", mv.version)
    mlflow.log_param("model_version", mv.version)


2025/05/26 17:20:34 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Omkar\AppData\Local\Temp\tmpx3jcngtt\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.6.1', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 
2025/05/26 17:20:34 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/05/26 17:20:44 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Omkar\AppData\Local\Temp\tmpimpk7mp6\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.6.1', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 
2025/05/26 17:20:44 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without

## Ridge Regression

In [27]:
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_absolute_error
import mlflow
import mlflow.sklearn

mlflow.set_experiment("optimizing-ad-spend-experiment")

# assume X_log and y_log are defined as before
with mlflow.start_run(run_name="Ridge_Regression_Log-Log"):
    alpha = 0.1
    ridge_model = Ridge(alpha=alpha)
    ridge_model.fit(X_log, y_log)

    y_pred = ridge_model.predict(X_log)
    r2 = r2_score(y_log, y_pred)
    mae = mean_absolute_error(y_log, y_pred)
    mlflow.log_metrics({"r_squared": r2, "mae_log": mae})

    mlflow.log_params({
        "model_type": "Ridge",
        "alpha": alpha,
        "transformation": "log1p",
        "n_features": X_log.shape[1]
    })

    coef_df = (
        pd.DataFrame({
            "feature": X_log.columns,
            "coefficient": ridge_model.coef_
        })
        .sort_values(by="coefficient", ascending=False)
    )
    for _, row in coef_df.iterrows():
        mlflow.log_metric(f"coef__{row['feature']}", row["coefficient"])

    os.makedirs("models", exist_ok=True)
    coef_path = "models/ridge_coefficients.csv"
    coef_df.to_csv(coef_path, index=False)
    mlflow.log_artifact(coef_path, artifact_path="coefficients")

    mlflow.sklearn.log_model(
        ridge_model,
        artifact_path="model",
        registered_model_name="Ridge_LogLog_Model"
    )

    run_id = mlflow.active_run().info.run_id
    mv = mlflow.register_model(f"runs:/{run_id}/model", "Ridge_LogLog_Model")
    mlflow.set_tag("model_version", mv.version)
    mlflow.log_param("model_version", mv.version)


2025/05/26 17:21:03 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Omkar\AppData\Local\Temp\tmpr9egk698\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.6.1', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 
2025/05/26 17:21:03 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/05/26 17:21:12 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Omkar\AppData\Local\Temp\tmpzvfgl_xe\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.6.1', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 
2025/05/26 17:21:12 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without

In [17]:
from sklearn.ensemble import RandomForestRegressor
mlflow.set_experiment("optimizing-ad-spend-experiment")

# Start MLflow run 
with mlflow.start_run(run_name="Random Forest Regressor"):
    # Train the first Random Forest Regressor model
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X, y)
    
    # Extract feature importances from the first model
    feature_importances = pd.DataFrame({
        'Feature': X.columns,
        'Importance': rf_model.feature_importances_
    }).sort_values(by='Importance', ascending=False)

    # Logging feature importances 
    for feature, importance in zip(feature_importances['Feature'], feature_importances['Importance']):
        mlflow.log_metric(f"importance_{feature}", importance) 

    # Train the second Random Forest Regressor model
    rf_model_1 = RandomForestRegressor(n_estimators=30, random_state=42)
    rf_model_1.fit(X, y)
    
    # Extract feature importances from the second model
    feature_importances_1 = pd.DataFrame({
        'Feature': X.columns,
        'Importance': rf_model_1.feature_importances_
    }).sort_values(by='Importance', ascending=False)

    # Logging feature importances for the second model
    for feature, importance in zip(feature_importances_1['Feature'], feature_importances_1['Importance']):
        mlflow.log_metric(f"importance_{feature}_model_2", importance)  
    
    # Save the feature importances DataFrame from the first model as a CSV and log it as an artifact
    coefficients_file_path = "models/rf_model_1_importances.csv"
    feature_importances.to_csv(coefficients_file_path, index=False)  # Saving feature importances to CSV
    
    # Log the CSV file from the first model as an artifact (will store it under the run's artifact directory)
    mlflow.log_artifact(coefficients_file_path)  # Log the CSV file as an artifact
    
    # Save the feature importances DataFrame from the second model as a CSV and log it as an artifact
    coefficients_file_path_1 = "models/rf_model_2_importances.csv"
    feature_importances_1.to_csv(coefficients_file_path_1, index=False)  # Saving feature importances to CSV
    
    # Log the CSV file from the second model as an artifact (will store it under the run's artifact directory)
    mlflow.log_artifact(coefficients_file_path_1)  # Log the CSV file as an artifact
    
    # Logging the models with wrong version names
    mlflow.log_param("model_type", "random_forest")  # Wrong model type as parameter
    mlflow.sklearn.log_model(rf_model, "model_1", registered_model_name="RandomForest_v1")  
    mlflow.sklearn.log_model(rf_model_1, "model_2", registered_model_name="RandomForest_v2")  


2025/05/26 15:26:01 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/05/26 15:26:01 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'RandomForest_v1'.
Created version '1' of model 'RandomForest_v1'.
2025/05/26 15:26:25 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/05/26 15:26:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'RandomForest_v2'.
Created version '1' of model 'RandomForest_v2'.


In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [30]:
import os
import pandas as pd
import mlflow
import mlflow.sklearn
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

mlflow.set_experiment("optimizing-ad-spend-experiment")

# Hyperparameter search space
search_space = {
    "n_estimators": hp.choice("n_estimators", [50, 100, 150, 200]),
    "max_depth":   hp.choice("max_depth",   [None, 10, 20, 30]),
    "min_samples_split": hp.choice("min_samples_split", [2, 5, 10]),
    "min_samples_leaf":  hp.choice("min_samples_leaf",  [1, 2, 4]),
}

def objective(params):
    with mlflow.start_run(nested=True):
        # Log hyperparameters
        mlflow.log_params(params)
        mlflow.log_param("model_type", "RandomForestRegressor")
        mlflow.log_param("random_state", 42)

        # Train & predict
        model = RandomForestRegressor(**params, random_state=42)
        model.fit(X_train, y_train)
        preds = model.predict(X_test)

        # Compute & log metrics
        mse = mean_squared_error(y_test, preds)
        r2  = r2_score(y_test, preds)
        mae = mean_absolute_error(y_test, preds)
        mlflow.log_metrics({"mse": mse, "r2": r2, "mae": mae})

        # Feature importances
        fi = (
            pd.DataFrame({
                "feature": X_train.columns,
                "importance": model.feature_importances_
            })
            .sort_values("importance", ascending=False)
        )
        for feat, imp in zip(fi["feature"], fi["importance"]):
            mlflow.log_metric(f"importance__{feat}", imp)

        # Name artifact by run ID
        run_id = mlflow.active_run().info.run_id
        os.makedirs("models", exist_ok=True)
        imp_path = f"models/rf_imp_{run_id}.csv"
        fi.to_csv(imp_path, index=False)
        mlflow.log_artifact(imp_path, artifact_path="feature_importances")

        # Log & register model
        mlflow.sklearn.log_model(
            model,
            artifact_path="model",
            registered_model_name="RandomForest_Optimized"
        )
        mv = mlflow.register_model(f"runs:/{run_id}/model", "RandomForest_Optimized")
        mlflow.set_tag("model_version", mv.version)
        mlflow.log_param("model_version", mv.version)

        return {"loss": mse, "status": STATUS_OK}

# Run Hyperopt
trials = Trials()
best = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials
)

# Map choice indices back to actual values
best_params = {
    "n_estimators": [50, 100, 150, 200][best["n_estimators"]],
    "max_depth":   [None, 10, 20, 30][best["max_depth"]],
    "min_samples_split": [2, 5, 10][best["min_samples_split"]],
    "min_samples_leaf":  [1, 2, 4][best["min_samples_leaf"]],
}

with mlflow.start_run(run_name="Best_RF_Model"):
    mlflow.log_params(best_params)
    final_model = RandomForestRegressor(**best_params, random_state=42)
    final_model.fit(X, y)
    preds_full = final_model.predict(X)
    mlflow.log_metrics({
        "r2_full": r2_score(y, preds_full),
        "mae_full": mean_absolute_error(y, preds_full)
    })
    mlflow.sklearn.log_model(
        final_model,
        artifact_path="model",
        registered_model_name="RandomForest_Optimized"
    )
    run_id = mlflow.active_run().info.run_id
    mv = mlflow.register_model(f"runs:/{run_id}/model", "RandomForest_Optimized")
    mlflow.set_tag("model_version", mv.version)
    mlflow.log_param("model_version", mv.version)

print("Best hyperparameters:", best_params)


  0%|                                                                                   | 0/50 [00:00<?, ?trial/s, best loss=?]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.002001 seconds
INFO:hyperopt.tpe:TPE using 0 trials
2025/05/26 17:27:50 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Omkar\AppData\Local\Temp\tmp3qunh78d\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.6.1', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 

2025/05/26 17:27:50 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.

2025/05/26 17:27:59 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Omkar\AppData\Local\Temp\tmpeoplaxq1\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.6.1', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 

2025/05/26 17:27:59 WARNING mlflow.utils.environmen

  2%|█                                                     | 1/50 [00:18<15:28, 18.95s/trial, best loss: 0.0005479834838118523]

Created version '1' of model 'RandomForest_Optimized'.
Registered model 'RandomForest_Optimized' already exists. Creating a new version of this model...
Created version '2' of model 'RandomForest_Optimized'.
INFO:hyperopt.tpe:build_posterior_wrapper took 0.003503 seconds
INFO:hyperopt.tpe:TPE using 1/1 trials with best loss 0.000548
2025/05/26 17:28:09 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Omkar\AppData\Local\Temp\tmp0vp2y26y\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.6.1', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 

2025/05/26 17:28:09 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.

2025/05/26 17:28:18 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Omkar\AppDat

  4%|██                                                   | 2/50 [00:38<15:13, 19.04s/trial, best loss: 0.00044333041972760014]

...

Created version '84' of model 'RandomForest_Optimized'.
INFO:hyperopt.tpe:build_posterior_wrapper took 0.001938 seconds
INFO:hyperopt.tpe:TPE using 42/42 trials with best loss 0.000443
2025/05/26 17:42:45 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Omkar\AppData\Local\Temp\tmpu8af_l6n\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.6.1', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 

2025/05/26 17:42:45 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.

2025/05/26 17:42:59 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\Omkar\AppData\Local\Temp\tmp810dpnff\model\model.pkl, flavor: sklearn). Fall back to return ['scikit-learn==1.6.1', 'cloudpickle==3.1.1']. Set logging level to DE

 86%|████████████████████████████████████████████▋       | 43/50 [15:19<02:57, 25.34s/trial, best loss: 0.00044333041972760014]

Created version '86' of model 'RandomForest_Optimized'.
INFO:hyperopt.tpe:build_posterior_wrapper took 0.001801 seconds
INFO:hyperopt.tpe:TPE using 43/43 trials with best loss 0.000443


 86%|████████████████████████████████████████████▋       | 43/50 [15:30<02:31, 21.63s/trial, best loss: 0.00044333041972760014]


KeyboardInterrupt: 